In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import explode,col,expr,array,when
from pyspark.sql.types import ArrayType, IntegerType, ShortType

In [3]:
spark = SparkSession.builder.appName("task1")
spark = SparkSession.builder.appName('Network').config("spark.driver.memory", "4g").getOrCreate()

25/05/09 11:22:04 WARN Utils: Your hostname, zaki-desktop resolves to a loopback address: 127.0.1.1; using 10.10.42.184 instead (on interface enp2s0)
25/05/09 11:22:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 11:22:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/09 11:22:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark

In [5]:
df_pyspark = spark.read.option('multiline','true').json('rate.json')

In [6]:
df_pyspark.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- billing_code_type_version: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negotiated_rates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- negotiated_prices: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- additional_information: string (nullable = true)
 |    |    |    |    |-- billing_class: string (nullable = true)
 |    |    |    |    |-- billing_code_modifier: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- expiration_date: string (nullable = true)
 |    |    |    |    |-- negotiated_rate: double (nullable = true)
 |    |    |    |    |-- negotiated_type: string (nullable = true)
 |    |    |    |    |-- service_code: array (nullable = true)
 |    |    |

In [7]:
negotiated_rates_df = df_pyspark.withColumn("row1",explode("negotiated_rates"))
negotiated_new = negotiated_rates_df.withColumn("id",explode("row1.provider_references"))
negotiated_prices_df = negotiated_new.withColumn("row2",explode("row1.negotiated_prices"))

rate_df = negotiated_prices_df.select("billing_code","billing_code_type","negotiation_arrangement",
                                      col("row2.billing_code_modifier").alias("billing_code_modifier"),
                                      col("row2.billing_class").alias("billing_class"),
                                      col("row2.negotiated_rate").alias("negotiated_rate"),
                                      col("row2.service_code").alias("service_code"),
                                      col("id").alias("provider_group_id"),
                                      col("row2.negotiated_type").alias("negotiated_type"))
rate_df.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- billing_class: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- service_code: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)



In [8]:
rate_df.show(10)

+------------+-----------------+-----------------------+---------------------+-------------+---------------+------------+-----------------+---------------+
|billing_code|billing_code_type|negotiation_arrangement|billing_code_modifier|billing_class|negotiated_rate|service_code|provider_group_id|negotiated_type|
+------------+-----------------+-----------------------+---------------------+-------------+---------------+------------+-----------------+---------------+
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           70.0|        [11]|         10001001|     negotiated|
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           70.0|        [11]|         10002001|     negotiated|
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           70.0|        [11]|         10003001|     negotiated|
|       PEMG1|              CPT|                    ffs|        

In [9]:
rate_df.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- billing_class: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- service_code: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)



In [10]:
net_cast = rate_df.withColumn("service_code", col("service_code").cast(ArrayType(IntegerType())))

In [11]:
net_cast.write.parquet('net_output.parquet')

In [12]:
net_cast.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- billing_class: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- service_code: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)

